<a href="https://colab.research.google.com/github/alexcontarino/personal-projects/blob/main/Sports_Betting/Profit_Boost_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The `boost_analysis` function of the code below finds threshold values for odds, boost value, or number of parlay legs to ensure a positive expectation for the profit of the bet, assuming some profit boost (10%, 20%, etc) is available.

## Example Uses:

1. **Find the right odds**: Given a 20% profit boost ($b=1.2$), assummed vig of 5% ($v=1.05$) and list of possible number of legs for a parlay ($l$), find the maximum true probability ($q$) to achieve a positive expected value:
`boost_analysis(b=1.2, v=1.05, l=range(1,6))`

1. **Find the minimum boost**: Given a desired true win probability of 0.25 ($q=0.25$), assummed vig of 5% ($v=1.05$) and list of number of legs for a possible parlay ($l$), find the minimum profit boost needed ($b$) to achieve a positive expected value: `boost_analysis(q=0.25, v=1.05, l=range(1,9))`

1. **Find the maximum number of legs**: Given a disired true win probability of 0.25, a profit boost of 33%, and assummed vig of 5% ($v=1.05$), find the maximum number of legs that can be added to the parlay and still achieve a postive expected value: `boost_analysis(q=0.10, b=1.33, v=1.05)`

## Table of Contents
1. [Full Explanation](#Full-Explanation)
1. [Set-Up & Helper Functions](#Set-Up-&-Helper-Functions)
1. [Main Function](#Main-Function)
1. [Example Uses](#Example-Uses)

# Full Explanation

In sports betting, the bettor wagers a stake $C$ and receives a profit $P$ of either:

$C \frac{1 - qv}{qv}$ with probability $q$, or

$-C$ with probability $1-q$

Where $q$ is the true probability of the bet "hitting," and $v$ is the "vig" charged by the sportsbook, typically around 5% (i.e., $v = 1.05$).

In a parlay, the bettor places multiple bets and receives a payout only if each bet, or "leg", of the parlay hits. For parlays, I assume that a constant vig is charged for each leg, with the bettor winning a payout:

$C \frac{1-qv^l}{qv^l}$

where $q$ is the true probability of the parlay winning and $l$ is the number of legs in the parlay. This construct allows for easy generalization.

The expected profit $E[P]$ from a bet is then:

$ E[P] = q [C\frac{1 - qv^l}{qv^l}] + (1-q)(-C)$

$ E[P] = \frac{C}{v^l} - C $

Note that, because $v>1$ and $l$ is a positive integer, $E[P] < 0$.

However, sportsbooks regurarly offer a "profit boost", in which the bettor's payout for a successful bet is increased by some percentage. For example, a \$10 wager on a bet with +100 odds and a 20% profit boost would payout \$22 instead of \$20. If we define $b$ as the percentage boost (i.e., $b=1.2$ implies a 20% boost), then the profit of a successful bet can be denoted as:

$ b C \frac{1-qv^l}{qv^l}$

and the expectation of profit as:

$ E[P] = b q [C \frac{1-qv^l}{qv^l}] - (1-q)(-C) $

The addition of the boost means $E[P]$ is no longer strictly less than 0, and an interesting question becomes finding the values for $b$, $q$ and $l$ which satisfy the equation:

$ 0 < b q [C \frac{1-qv^l}{qv^l}] - (1-q)(-C) $

Re-arranging the terms above, we arrive at threshold values for $b$, $q$ and $l$ to ensure a positive expected value when given the other two (and assuming some constant value for $v$):

- $ q < \frac{b - v^l}{v^l(b-1)} $

- $ b > \frac{v^l(1-q)}{1-v^lq} $

- $ l < \lfloor \frac{ln(b) - ln(1-q+bq)}{ln(v)} \rfloor $

Of course, the bettor's desire is to *make money*, not just break even. We can generalize the expressions above by defining $M$, a desired minimum expected profit as a percentage of the wager amount $C$:

$M = \frac{E[P]}{C} $

$q$, $b$, and $l$ then must satisfy:

$ M = \frac{E[P]}{C} \leq b q \frac{1-qv^l}{qv^l} - (1-q)(-1) $

In this general case, the thresholds of our key variables are:

- $ q < \frac{b - v^l(M+1)}{v^l(b-1)} $

- $ b > \frac{v^l(M+1-q)}{1-v^lq} $

- $ l < \lfloor \frac{ln(b) - ln(M+1-q+bq)}{ln(v)} \rfloor $

`boost_analysis()` and its helper functions below implement finding these thresholds using a defined vig and mimimum expected value of profit to pursue.

# Set-Up & Helper Functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
def prob_to_odds(prb, add_vig=False, vig=1.05):

  if prb is None:
    return None

  if add_vig:
    prb = prb * vig

  if prb > 0.5:
    odds = math.floor(-100 * prb / (1 - prb))
    return "{}".format(odds)
  else:
    odds = math.ceil(100 * (1 - prb) / prb)
    return "+{}".format(odds)

In [3]:
def odds_per_leg(offered_prb, legs):

  if offered_prb is None:
    return None

  prb_per_leg = offered_prb ** (1/legs)

  return prob_to_odds(prb_per_leg)

In [4]:
def profit_calculator(q, b, l, v):

  if q*v**l < 0 or q*v**l > 1:
    return None

  return b*(1-q*v**l)/v**l - 1 + q

In [5]:
def probability_formatter(prb,digits):

  try:
    if np.abs(prb) < 0.00001:
      return "0.00%"
    else:
      return "{}%".format(round(100*prb,digits))
  except:
    return None

# Main Function

In [6]:
def boost_analysis(q=None, b=None, l=None, v=1.05, M=0.0):
  '''
  INPUTS: Function expects to receive 2 of q, b, and l in order to compute the other; v and expected_profit are required inputs with default values as shown
  --q: minimum true probability to achieve positive expected value (typically unknown). If not none, function expects a decimal value (or list of such values) between 0 and 1 (75% = 0.75)
  --b: payout boost offered by the sportsbook (typically known). If not none, function expects a decimal value greater than 1 (20% boost = 1.2; 105% boost = 2.05; etc.)
  --l: number of legs in the bet. If not none, function expects a single positive integer or list/array of such values.
  --v: sportbook's vig (typically assumed). Function expects this as a decimal value greater than 1 (5% vig = 1.05). Default is 1.05
  --min_ep: minimum expected profit to pursue. Function expects a number value. Default is 0.
  OUTPUT: dataframe that contains the following columns. Column names are
  --legs (l): Given number of legs in bet or maximum legs allowed to achieve positive expected value
  --boost (b): given boost or minimum boost to achieve positive expected value
  --vig (v): assumed vig
  --true_prb (q): given true probability or maximum true probability allowable to achieve positive expected value. Infeasible (negative) values are allowed to be computed for user's reference
  --offered_prb: offered probability computed as q * v**l. If the true_prb is negative, the offered_prb will be None
  --offered_odds: offered probability converted to American odds. If the true_prb is negative, the offered_odds will be None
  --odds_per_leg: the odds per leg of bet to achieve total offered odds, assuming independence of each leg. If the true_prb is negative, the odds_per_leg will be None.
  --expected_profit:
  '''

  if q is None:

    return_dict = {"boost": [],
                   "vig": [],
                   "max_true_prb": [],
                   "max_offered_prb": [],
                   "min_offered_odds": [],
                   "legs": [],
                   "odds_per_leg": [],
                   "expected_profit_prct": []}

    l = np.array(l).flatten()
    for legs in l:
      return_dict["legs"].append(legs)
      q_i = (b - v**legs*(1+M)) / (b - 1) / v**legs
      offered_prb = q_i * v**legs
      if offered_prb > 1 or offered_prb < 0:
        offered_prb = None
      offered_odds = prob_to_odds(offered_prb)
      return_dict["min_offered_odds"].append(offered_odds)
      return_dict["max_true_prb"].append(q_i)
      return_dict["max_offered_prb"].append(offered_prb)
      return_dict["boost"].append(probability_formatter(b-1,0))
      return_dict["vig"].append(probability_formatter(v-1,0))
      return_dict["odds_per_leg"].append(odds_per_leg(offered_prb, legs))
      return_dict["expected_profit_prct"].append(probability_formatter(profit_calculator(q_i, b, legs, v),2))

    df = pd.DataFrame(return_dict)

    return df.set_index("legs")


  if b is None:

    return_dict = {"min_boost": [],
                   "vig": [],
                   "true_prb": [],
                   "offered_prb": [],
                   "offered_odds": [],
                   "legs": [],
                   "odds_per_leg": [],
                   "expected_profit_prct": []}

    l = np.array(l).flatten()
    q = np.array(q).flatten()
    for legs in l:
      for q_i in q:
        return_dict["legs"].append(legs)
        min_boost = v**legs*(M+1-q_i) / (1 - v**legs * q_i)
        offered_prb = q_i * v**legs
        offered_odds = prob_to_odds(offered_prb)
        return_dict["offered_odds"].append(offered_odds)
        return_dict["true_prb"].append(q_i)
        return_dict["offered_prb"].append(offered_prb)
        return_dict["min_boost"].append(probability_formatter(min_boost-1,0))
        return_dict["vig"].append(probability_formatter(v-1,0))
        return_dict["odds_per_leg"].append(odds_per_leg(offered_prb, legs))
        return_dict["expected_profit_prct"].append(probability_formatter(profit_calculator(q_i, min_boost, legs, v),2))


    df = pd.DataFrame(return_dict)

    return df.set_index("legs")


  if l is None:

    return_dict = {"boost": [],
                   "vig": [],
                   "true_prb": [],
                   "offered_prb": [],
                   "offered_odds": [],
                   "legs": [],
                   "odds_per_leg": [],
                   "expected_profit_prct": []}

    max_legs = (math.log(b) - math.log(M+1-q+b*q)) / math.log(v)

    if math.floor(max_legs) < 1:
      print("Maximum legs found: {}".format(max_legs))
      print("No legs allow for positive expected profit under given conditions")
      return None

    l = np.arange(1, math.floor(max_legs) + 1)
    q = np.array(q).flatten()

    for legs in l:
      for q_i in q:
        offered_prb = q_i * v**legs
        offered_odds = prob_to_odds(offered_prb)
        return_dict["legs"].append(legs)
        return_dict["offered_odds"].append(offered_odds)
        return_dict["true_prb"].append(q_i)
        return_dict["offered_prb"].append(offered_prb)
        return_dict["boost"].append(probability_formatter(b-1,0))
        return_dict["vig"].append(probability_formatter(v-1,0))
        return_dict["odds_per_leg"].append(odds_per_leg(offered_prb, legs))
        return_dict["expected_profit_prct"].append(probability_formatter(profit_calculator(q_i, b, legs, v),2))

    df = pd.DataFrame(return_dict)

    return df.set_index("legs")


# Example Uses

Given a 20% profit boost ($b=1.20$), assummed vig of 5% ($v=1.05$) and list of possible number of legs for a parlay ($l$), find the maximum true probability ($q$) to achieve a positive expected value. **(Typical use. Bettor is given a profit boost and must construct a bet or parlay meeting certain criteria: minimum legs, minimum total odds, etc.)**


In [7]:
boost_analysis(b=1.20, v=1.05, l=range(1,6), M=0.00)

,boost,vig,max_true_prb,max_offered_prb,min_offered_odds,odds_per_leg,expected_profit_prct
legs,,,,,,,
1,20.0%,5.0%,0.714286,0.750000,-300,-300,0.00%
2,20.0%,5.0%,0.442177,0.487500,+106,-232,0.00%
3,20.0%,5.0%,0.183026,0.211875,+372,-148,0.00%
4,20.0%,5.0%,-0.063785,NaN,None,None,None
5,20.0%,5.0%,-0.298843,NaN,None,None,None


Given desired true win probability of 0.25 ($q=0.25$), assummed vig of 5% ($v=1.05$) and list of possible number of legs for a parlay ($l$), find the minimum profit boost needed ($b$) to achieve a positive expected value. **(Useful when the amount of profit boost depends on the number of parlay legs--e.g., DraftKings' "Stepped-up Parlay".)**


In [8]:
boost_analysis(q=0.01, v=1.05, l=range(1,12), M=0.00)

,min_boost,vig,true_prb,offered_prb,offered_odds,odds_per_leg,expected_profit_prct
legs,,,,,,,
1,5.0%,5.0%,0.01,0.010500,+9424,+9424,0.00%
2,10.0%,5.0%,0.01,0.011025,+8971,+853,0.00%
3,16.0%,5.0%,0.01,0.011576,+8539,+343,0.00%
4,22.0%,5.0%,0.01,0.012155,+8128,+202,0.00%
5,28.0%,5.0%,0.01,0.012763,+7736,+140,0.00%
6,34.0%,5.0%,0.01,0.013401,+7363,+106,0.00%
7,41.0%,5.0%,0.01,0.014071,+7007,-120,0.00%
8,48.0%,5.0%,0.01,0.014775,+6669,-145,0.00%
9,56.0%,5.0%,0.01,0.015513,+6347,-170,0.00%


**Find the maximum number of legs**: Given a disired true win probability of 0.25, a profit boost of 33%, and assummed vig of 5% ($v=1.05$), find the maximum number of legs that can be added to the parlay. **(Useful if better is attempting to reach a certain payout value or total odds.)**  

In [9]:
boost_analysis(q=0.25, b=1.33, v=1.05, M=0.00)

,boost,vig,true_prb,offered_prb,offered_odds,odds_per_leg,expected_profit_prct
legs,,,,,,,
1,33.0%,5.0%,0.25,0.262500,+281,+281,18.42%
2,33.0%,5.0%,0.25,0.275625,+263,-111,12.38%
3,33.0%,5.0%,0.25,0.289406,+246,-196,6.64%
4,33.0%,5.0%,0.25,0.303877,+230,-289,1.17%
